In [1]:
from libraries import *
from functions import *
from fitting import *
from util import *
%matplotlib qt

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


Current progress: 50000
Current progress: 100000
Current progress: 150000
Current progress: 200000
Current progress: 250000
DONE


### Plot result from pickles

Given an index and a channel, plot the spectra of the channel along with the fitted functions

In [ ]:
def plot_result(entry, channel):
    entry = data[entry]
    #print(entry['regular_fit']['powerlaw'])
    fig, axs = plt.subplots(2, 2)
    #print('powerlaw' in entry)
    if 'powerlaw' in entry['regular_fit']:
        axs[0,0].plot(vims_wave, entry['spectrum']-powerlaw_no_constant(vims_wave, *entry['regular_fit']['powerlaw'].values()), label = "Spectra (powerlaw adjusted)")
        axs[0,0].plot(vims_wave, gaussian(vims_wave, *entry['regular_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[0,0].plot(vims_wave, skew(vims_wave, *(entry['regular_fit'][channel]['skew'][x] for x in entry['regular_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
        
    else:
        axs[0,0].plot(vims_wave, entry['spectrum'], label = "Spectrum (no powerlaw adjustment)")
        axs[0,0].plot(vims_wave, gaussian(vims_wave, *entry['regular_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[0,0].plot(vims_wave, skew(vims_wave, *(entry['regular_fit'][channel]['skew'][x] for x in entry['regular_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
    
    axs[0,1].plot(vims_wave, entry['stefan_spectrum'], label = "Stefan")
    axs[0,1].plot(vims_wave, gaussian(vims_wave, *entry['stefan_fit'][channel]['gaussian'].values()), label = "Gaussian")
    axs[0,1].plot(vims_wave, skew(vims_wave, *(entry['stefan_fit'][channel]['skew'][x] for x in entry['stefan_fit'][channel]['skew'] if 'peak' not in x)), label = "Skew")
    
    axs[0,0].legend()
    axs[0,1].legend()
    plt.show()
    #plt.plot(vims_wave,entry['spectrum'], label = 'Original')
    #plt.plot(vims_wave,entry['stefan_spectrum'], label = 'Stefan')
    #plt.plot()

plot_result(72, '2.0')

In [3]:
filtered_data = filter_data(data)

{'name': 'V1753535313_1.ir_rc20final.cub', 'sequenceid': 'S79', 'observationid': 'VIMS_195TI_HIRES003_PRIME', 'latitude': -14.192410526012, 'longitude': 113.08447714133, 'phase': 79.247331641247, 'incidence': 68.628679969023, 'emission': 26.016340587574, 'distance': 7368.7025357152, 'lineres': 1965.8545564098, 'sampres': 2466.4575678517, 'scposx': -2735.0425878636, 'scposy': 5677.0692141084, 'scposz': -3819.4507773567, 'sunposx': -1380094499.4225, 'sunposy': 147413363.40772, 'sunposz': 488789535.78307, 'ettime': 428112953.54426, 'starttime': '2013-07-26 12:12:41.994', 'exposureduration': 160.0, 'spectrum': array([ 0.0404514 ,  0.0663658 ,  0.086725  ,  0.0854239 ,  0.0644469 ,
        0.0462576 ,  0.0390803 ,  0.0374057 ,  0.048135  ,  0.0629884 ,
        0.0766593 ,  0.0830114 ,  0.0691124 ,  0.0468681 ,  0.0240458 ,
        0.0202946 ,  0.0128574 ,  0.016177  ,  0.021298  ,  0.032576  ,
        0.0397113 ,  0.0422687 ,  0.0508583 ,  0.059501  ,  0.0663442 ,
        0.042854  ,  0.026

Total 89372 entries removed


In [ ]:
len(filtered_data)

In [9]:
buckets = time_bucket(filtered_data)
[(x, len(buckets[x])) for x in buckets]

[((152047486.19852, 170097356.313667), 3465),
 ((170097356.313667, 188147226.428814), 1211),
 ((188147226.428814, 206197096.543961), 3524),
 ((206197096.543961, 224246966.65910798), 11486),
 ((224246966.65910798, 242296836.77425498), 40716),
 ((242296836.77425498, 260346706.88940197), 11958),
 ((260346706.88940197, 278396577.00454897), 1480),
 ((278396577.00454897, 296446447.11969596), 3327),
 ((296446447.11969596, 314496317.23484296), 10047),
 ((314496317.23484296, 332546187.34998995), 31004),
 ((332546187.34998995, 350596057.46513695), 8044),
 ((350596057.46513695, 368645927.58028394), 6366),
 ((368645927.58028394, 386695797.69543093), 14053),
 ((386695797.69543093, 404745667.8105779), 6333),
 ((404745667.8105779, 422795537.9257249), 14100),
 ((422795537.9257249, 440845408.0408719), 18262),
 ((440845408.0408719, 458895278.1560189), 406),
 ((458895278.1560189, 476945148.2711659), 1174),
 ((476945148.2711659, 494995018.3863129), 1972),
 ((494995018.3863129, 513044888.5014599), 1491)]

### Rainbow Plots

Using the data split into time buckets, plot one gaussian for each time bucket to achieve a rainbow-like effect.

![Two-micron rainbow](rainbow.png "Rainbow")

In [31]:
def plot_average_fits_rainbow(buckets, channel = '2.0', fit_type = 'regular_fit'):
    ncolors = len(buckets)
    colors = plt.cm.nipy_spectral(np.linspace(0.05,0.95,ncolors))
    
    parameters = {x: {} for x in buckets}
    
    for i, bucket in enumerate(buckets):
        mean = []
        amplitudes = []
        width = []
        
        for entry in buckets[bucket]:
            amplitudes.append(entry[fit_type][channel]['gaussian']['amp'])
            width.append(entry[fit_type][channel]['gaussian']['sigma'])
            mean.append(entry[fit_type][channel]['gaussian']['mean'])
        
        median = len(mean) // 2
        
        parameters[bucket]['mean'] = mean[median]
        parameters[bucket]['width'] = width[median]
        parameters[bucket]['amplitudes'] = amplitudes[median]
        
    for i, bucket in enumerate(parameters):
        if parameters[bucket].get('mean', None) != None:
            mean_, sigma_, amp_ = parameters[bucket]['mean'], parameters[bucket]['width'], parameters[bucket]['amplitudes']
            x = np.linspace(0, 5, 1000)
            plt.plot(x, gaussian(x, mean_, , 1), color = colors[i], label = bucket)

    plt.legend()

In [32]:
plot_average_fits_rainbow(buckets, channel = '2.0', fit_type = 'stefan_fit')